In [4]:
import scipy.io as sio
import numpy as np

from __future__ import print_function

import keras.callbacks as cb
import keras
from keras.layers import Input, Activation, Dense, Dropout, Flatten, merge
from keras.layers import Convolution3D, MaxPooling3D
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adadelta
from keras.regularizers import l1, l2
from keras.utils import np_utils
from keras import layers
from keras import backend as K  
from IPython.core.debugger import Tracer
from random import shuffle

%matplotlib inline
from matplotlib import pyplot as plt
import time

import pickle


Using TensorFlow backend.


In [5]:
pkl_file = open('C:\\Users\\PangTouXian\\Documents\\Python_scripts\\cnnChair\\data\\cubeTest16_1dir', 'rb')
cubeTest16 = pickle.load(pkl_file)
pkl_file.close()
pkl_file = open('C:\\Users\\PangTouXian\\Documents\\Python_scripts\\cnnChair\\data\\cubeTrain16_1dir', 'rb')
cubeTrain16 = pickle.load(pkl_file)
pkl_file.close()
cubeDim = 16

In [6]:
def PreprocessDataset():
    # Load dataset
    trainData = cubeTrain16
    testData = cubeTest16
    trainLabel =np.concatenate((np.zeros(106),np.ones(515),2*np.ones(889),3*np.ones(200),4*np.ones(200),
                                5*np.ones(465),6*np.ones(200),7*np.ones(680),8*np.ones(392),9*np.ones(344)))
    testLabel =np.concatenate((np.zeros(50),np.ones(100),2*np.ones(100),3*np.ones(86),4*np.ones(86),
                               5*np.ones(100),6*np.ones(86),7*np.ones(100),8*np.ones(100),9*np.ones(100))) 
    print (trainData.shape)
    print(trainLabel.shape) 
    
    # Transform lables to one-hot
    print(trainLabel[0])
    trainLabel = np_utils.to_categorical(trainLabel, 10)
    testLabel = np_utils.to_categorical(testLabel, 10)
    print(trainLabel[0])
    print(trainLabel.shape)
    print(testLabel.shape)
    
    # Reshape
    trainData = trainData.reshape(trainData.shape[0], cubeDim,cubeDim,cubeDim, 1)
    testData = testData.reshape(testData.shape[0], cubeDim,cubeDim,cubeDim, 1)
    print(trainData.shape)
    return [trainData,testData,trainLabel,testLabel]

trainData,testData,trainLabel,testLabel = PreprocessDataset()

(3991, 16, 16, 16)
(3991,)
0.0
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(3991, 10)
(908, 10)
(3991, 16, 16, 16, 1)


In [7]:
def DefineModel_1():
    input = Input(shape=(cubeDim,cubeDim,cubeDim,1))
    
    conv1a = Convolution3D(64,3,3,3,strides=(1, 1, 1), activation='relu', 
                          padding='same')(input)
    conv1b = Convolution3D(64,2,2,2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(input)
    merged1 = merge([conv1a,conv1b],mode='concat',concat_axis=-1)
    print(np.shape(merged1))
    pooling1 = MaxPooling3D(pool_size=(2, 2, 2))(merged1)
    dropout = Dropout(0.5)(pooling1)
    
    conv2a = Convolution3D(64,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling1)
    conv2b = Convolution3D(128,3,3,3,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv2a)
    conv2c = Convolution3D(64,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling1)
    conv2d = Convolution3D(128,2,2,2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv2c)
    conv2e = Convolution3D(64,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling1) 
    merged2 = merge([conv2b,conv2d,conv2e],mode='concat',concat_axis=-1)
    print(np.shape(merged2))
    pooling2 = MaxPooling3D(pool_size=(2, 2, 2))(merged2)
    dropout = Dropout(0.5)(pooling2)
    
    conv3a = Convolution3D(128,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling2)
    conv3b = Convolution3D(256,3,3,3,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv3a)
    conv3c = Convolution3D(128,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling2)
    conv3d = Convolution3D(256,2,2,2,strides=(1, 1, 1), activation='relu', 
                          padding='same')(conv3c)
    conv3e = Convolution3D(128,1,1,1,strides=(1, 1, 1), activation='relu', 
                          padding='same')(pooling2)
    merged3 = merge([conv3b,conv3d,conv3e],mode='concat',concat_axis=-1)
    print(np.shape(merged3))
    flatten2 = Flatten()(merged3)    
    
    dense1 = Dense(256,activation='relu')(flatten2)
    dropout = Dropout(0.5)(dense1)
    dense2 = Dense(10, activation='softmax')(dropout)
    model = Model(inputs=input, outputs=dense2)
    
    optmzr = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])
 
    return model

model = DefineModel_1()
print(model.output_shape)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
  """
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
  import sys
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc

(?, 16, 16, 16, 128)
(?, 8, 8, 8, 320)
(?, 4, 4, 4, 640)
(None, 10)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\sit

In [8]:
def TrainModel_1(data=None, epochs=20, batch=128):
    start_time = time.time()
    model = DefineModel_1()
    if data is None:
        print("Must provide data.")
        return
    trainData,testData,trainLabel,testLabel = data
    print('Start training.')

    history = model.fit(trainData, trainLabel, nb_epoch=epochs, batch_size=batch,
              validation_data=(testData, testLabel),verbose=1,shuffle=True)
    print("Training took {0} seconds.".format(time.time() - start_time))
    return model, history

In [ ]:
trained_model_1, training_history_1 = TrainModel_1(data=[trainData,testData,trainLabel,testLabel])

C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
  """
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
  import sys
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc

(?, 16, 16, 16, 128)
(?, 8, 8, 8, 320)


C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (2, 2, 2), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (1, 1, 1), activation="relu", padding="same", strides=(1, 1, 1))`
C:\Users\PangTouXian\Anaconda3\envs\newTensorflowGpu\lib\sit

(?, 4, 4, 4, 640)
Start training.
Train on 3991 samples, validate on 908 samples
Epoch 1/20


In [ ]:
#original

num_filters1 = 64
num_filters2 = 128
filter_size1 = 3
filter_size2 = 2

def DefineModel_2():
    input = Input(shape=(cubeDim,cubeDim,cubeDim,1))
    conv1a = Convolution3D(num_filters1, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='valid',input_shape=(cubeDim,cubeDim,cubeDim, 1))(input)
    conv1b = Convolution3D(num_filters1, filter_size1,filter_size1,filter_size1, activation='relu')(conv1a)
    pooling1a = MaxPooling3D(pool_size=(2, 2, 2))(conv1b)
    print(np.shape(conv1a))    
    print(np.shape(input))
    conv1c = Convolution3D(num_filters2, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='valid')(pooling1a)
    conv1d = Convolution3D(num_filters2, filter_size1,filter_size1,filter_size1, strides=(1, 1, 1), activation='relu', 
                          padding='valid')(conv1c)
    pooling1b = MaxPooling3D(pool_size=(2, 2, 2))(conv1d)

    dropout1 = Dropout(0.5)(pooling1b)
    
    ''''''
    flatten1 = Flatten()(dropout1)
    
    conv2a = Convolution3D(num_filters1, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid',input_shape=(cubeDim,cubeDim,cubeDim, 1))(input)
    conv2b = Convolution3D(num_filters1, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid')(conv2a)
    pooling2a = MaxPooling3D(pool_size=(2, 2, 2))(conv2b)
    
    conv2c = Convolution3D(num_filters2, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid')(pooling2a)
    conv2d = Convolution3D(num_filters2, filter_size2, filter_size2, filter_size2,strides=(1, 1, 1), activation='relu', 
                          padding='valid')(conv2c)
    pooling2b = MaxPooling3D(pool_size=(2, 2, 2))(conv2d)
    dropout2 = Dropout(0.5)(pooling2b)
    
    
    ''''''
    flatten2 = Flatten()(dropout2)
    merged = merge([flatten1, flatten2],mode='concat',concat_axis=1,name='merged')
    
    '''merged = keras.layers.concatenate([dropout1, dropout2],axis=-1)'''

    '''flatten = Flatten()(merged)'''
    
    dense1 = Dense(128,activation='relu')(merged)
    dropout = Dropout(0.5)(dense1)
    dense2 = Dense(10, activation='softmax')(dropout)
    
    model = Model(inputs=input, outputs=dense2)
    
    ''' optmzr = SGD(lr=0.1, clipnorm=5.)''' '''--88%'''
    
    optmzr = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])
    '''
    
    optmzr = keras.optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)   
    model.compile(loss='categorical_crossentropy', optimizer=optmzr, metrics=["accuracy"])'''
    return model


model = DefineModel_2()
print(model.output_shape)


In [ ]:
def TrainModel_2(data=None, epochs=20, batch=128):
    start_time = time.time()
    model = DefineModel_2()
    if data is None:
        print("Must provide data.")
        return
    trainData,testData,trainLabel,testLabel = data
    print('Start training.')

    history = model.fit(trainData, trainLabel, nb_epoch=epochs, batch_size=batch,
              validation_data=(testData, testLabel),verbose=1,shuffle=True)
    print("Training took {0} seconds.".format(time.time() - start_time))
    return model, history

In [ ]:
trained_model_2, training_history_2 = TrainModel_2(data=[trainData,testData,trainLabel,testLabel])

In [ ]:
def TestModel(model=None, data=None):
    if model is None:
        print("Must provide a trained model.")
        return
    if data is None:
        print("Must provide data.")
        return
    
    result =model.predict_on_batch(np.reshape(data,[1,16,16,16,1]))
    return result

In [28]:
result = TestModel(trained_model, testData[0,:])
print(result)

[[9.9999940e-01 2.1265177e-07 3.6914465e-11 5.5789432e-11 3.3710421e-09
  1.6648455e-08 1.5302116e-09 2.6116270e-08 3.8751646e-09 3.5076536e-07]]
